In [6]:
from pybit.unified_trading import HTTP
from dotenv import load_dotenv
import pandas as pd
import os
import pprint
load_dotenv()



session = HTTP(
    testnet=False,
    demo=True,
    api_key=os.getenv('API_DEMO_KEY'),
    api_secret=os.getenv('API_DEMO_SECRET'),
)


def get_account_balance(ticker):
    # Retrieve account balance from Bybit
    response = session.get_wallet_balance(accountType="UNIFIED",coin=ticker)  

    
    if response['retCode'] == 0:
        balance = response['result']['list'][0]['coin'][0]['walletBalance']
        print(f'The balance of {ticker} is: {balance}')
        return balance
    else:
        print("Failed to retrieve account balance:", response['ret_msg'])
        return None


def fetch_market_data(symbol):
    data = session.get_kline(symbol=symbol, interval=1)

    df = pd.DataFrame(data['result']['list'])
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']
    df['Date'] = pd.to_datetime(pd.to_numeric(df['Date']), unit='ms')

    df.set_index('Date', inplace=True)

    df = df[::-1]
    return df


def place_order(symbol, side, qty_percentage, price, stop_loss=None, take_profit=None, leverage=10):
    balance = float(get_account_balance('USDT'))
    qty_before_lev = (balance*qty_percentage)/float(price)
    qty_final = round(qty_before_lev*leverage,3)
    print(session.place_order(
        category = "linear",
        symbol = symbol,
        side = side,
        orderType = "Limit",
        qty = str(qty_final),
        price=price,
        take_profit = take_profit,
        stop_loss = stop_loss,
        tpslMode = 'Full'

    ))


def calculate_indicators(df, window=14):
    #METHOD 1 RSI
    change = df["Close"].astype(float).diff()
    change.dropna(inplace = True)
    change_up = change.copy()
    change_down = change.copy()
    change_up[change_up<0]=0
    change_down[change_down>0]=0
    change.equals(change_up+change_down)
    avg_up = change_up.rolling(14).mean()
    avg_down = change_down.rolling(14).mean().abs()
    df['rsi'] = 100* avg_up/(avg_up+avg_down)


    #METHOD 2 RSI
    delta = df['Close'].astype(float).diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['rsi2'] = 100 - (100 / (1 + rs))
    df['rsi_sma'] = df['rsi'].rolling(window=14).mean()



    # MACD
    df['EMA12']= df['Close'].astype(float).ewm(span=12).mean() #SHORT EMA
    df['EMA26']= df['Close'].astype(float).ewm(span=26).mean()  #LONG EMA
    df['macd']= df['EMA12']-df['EMA26']
    df['macd_signal'] = df['macd'].ewm(span=9).mean()

    return df


    
symbol = 'BTCUSDT'
data = fetch_market_data(symbol)
new = calculate_indicators(data)
display(new)
'''
print(session.set_leverage(
    category="linear",
    symbol="BTCUSDT",
    buyLeverage="10",
    sellLeverage="10",
))
'''

get_account_balance('USDT')
#place_order('BTCUSDT', "Sell", 0.05, "110000", "120000", "100000", 10)
pprint.pp(session.get_open_orders(
    category="linear",
    symbol="BTCUSDT",
    openOnly=0,
    limit=1,
))


,Open,High,Low,Close,Volume,Turnover,rsi,rsi2,rsi_sma,EMA12,EMA26,macd,macd_signal
Date,,,,,,,,,,,,,
2025-01-21 11:26:00,103625.3,103697.1,103591.5,103689.2,55.483,5751266.7765,NaN,NaN,NaN,103689.200000,103689.200000,0.000000,0.000000
2025-01-21 11:27:00,103689.2,103700,103650.4,103693.2,79.947,8288928.7446,NaN,NaN,NaN,103691.366667,103691.276923,0.089744,0.049858
2025-01-21 11:28:00,103693.2,103826.7,103681.8,103787.3,178.872,18557234.7849,NaN,NaN,NaN,103728.809469,103725.777082,3.032387,1.272205
2025-01-21 11:29:00,103787.3,103794.6,103597.3,103648.3,123.914,12849819.8032,NaN,NaN,NaN,103703.395776,103704.117876,-0.722100,0.596628
2025-01-21 11:30:00,103648.3,103693.5,103607.1,103626.4,49.148,5094373.5011,NaN,NaN,NaN,103682.476299,103686.094780,-3.618481,-0.657272
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-21 14:41:00,103406.6,103430.9,103262.3,103357.8,251.412,25987523.807,13.561200,13.561200,37.709400,103906.958240,104265.656914,-358.698674,-201.320326
2025-01-21 14:42:00,103357.8,103858,103337.4,103732.9,600.651,62271925.1664,24.683427,24.683427,34.722784,103880.180049,104226.193429,-346.013379,-230.258937
2025-01-21 14:43:00,103732.9,103810,103576.6,103802,146.228,15157369.6412,22.835185,22.835185,31.686663,103868.152350,104194.771686,-326.619336,-249.531017


The balance of USDT is: 100070
{'retCode': 0,
 'retMsg': 'OK',
 'result': {'nextPageCursor': '1db872f6-9d72-4723-8b40-62eaa662f61d%3A1737372449550%2C1db872f6-9d72-4723-8b40-62eaa662f61d%3A1737372449550',
            'category': 'linear',
            'list': [{'symbol': 'BTCUSDT',
                      'orderType': 'Limit',
                      'orderLinkId': '',
                      'slLimitPrice': '0',
                      'orderId': '1db872f6-9d72-4723-8b40-62eaa662f61d',
                      'cancelType': 'UNKNOWN',
                      'avgPrice': '',
                      'stopOrderType': '',
                      'lastPriceOnCreated': '108326.2',
                      'orderStatus': 'New',
                      'createType': 'CreateByUser',
                      'takeProfit': '100000',
                      'cumExecValue': '0',
                      'tpslMode': 'Full',
                      'smpType': 'None',
                      'triggerDirection': 0,
                     

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf

def add_trading_signals(df):
    df['signal'] = 0

    for i in range(1, len(df)):
        if df['rsi'].iloc[i] < 30 and df['rsi'].iloc[i - 1] >= 30:
            df.loc[df.index[i], 'signal'] = 1  # Buy signal
        elif df['rsi'].iloc[i] > 70 and df['rsi'].iloc[i - 1] <= 70:
            df.loc[df.index[i], 'signal'] = -1  # Sell signal
    
    return df

# Apply trading signals to the data
data_with_signals = add_trading_signals(new)

# Display updated data
display(data_with_signals)

# Plot buy/sell signals on the RSI chart
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(data_with_signals.index, data_with_signals['rsi'], label='RSI', color='orange')
ax.axhline(30, linestyle='--', color='green', linewidth=1.5)
ax.axhline(70, linestyle='--', color='red', linewidth=1.5)

# Mark buy signals
buy_signals = data_with_signals[data_with_signals['signal'] == 1]
sell_signals = data_with_signals[data_with_signals['signal'] == -1]
ax.scatter(buy_signals.index, buy_signals['rsi'], marker='^', color='green', label='Buy Signal', alpha=1)
ax.scatter(sell_signals.index, sell_signals['rsi'], marker='v', color='red', label='Sell Signal', alpha=1)

ax.set_title('RSI with Buy and Sell Signals')
ax.legend()
plt.tight_layout()
plt.show()
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (16, 16)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=(16, 16))  



candlestick_data = data[['Open', 'High', 'Low', 'Close']].astype(float) 
mpf.plot(
    candlestick_data,
    type='candle',
    ax=ax1,
    style='charles',
    mav=(12,),  # Moving average, using 12-Day EMA
    volume=False, 
    show_nontrading=True
)
ax1.set_title(f'{symbol} Candlestick Chart with 12-Day EMA and Volume')


# Second chart: Plot the RSI
ax2.set_title('Relative Strength Index')
ax2.plot(data['rsi'], color='orange', linewidth=1)
ax2.axhline(30, linestyle='--', linewidth=1.5, color='green')  # Oversold level
ax2.axhline(70, linestyle='--', linewidth=1.5, color='red')    # Overbought level

#3
ax3.set_title('Relative Strength Index 2')
ax3.plot(data['rsi2'], color='orange', linewidth=1)
ax3.axhline(30, linestyle='--', linewidth=1.5, color='green')  # Oversold level
ax3.axhline(70, linestyle='--', linewidth=1.5, color='red')    # Overbought level


ax4.set_title('MACD and MACD Signal')
ax4.plot(data['macd'], label='MACD', color='blue', linewidth=1)
ax4.plot(data['macd_signal'], label='Signal Line', color='red', linewidth=1)
ax4.axhline(0, linestyle='--', color='black', linewidth=1)  
ax4.legend(loc='upper left')


ax3.set_ylim(0, 90)  



plt.tight_layout()  
plt.show()
